# First modelling tests 

## Data import and pretreatment

* Data pretreatment developed on file `eda_v1`. 
* Data is imported in 2 chunks - not possible to query so many fields at once in Supermetrics

In [1]:
import requests
import urllib.parse as up
import json
from pandas.io.json import json_normalize
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt 
import constants_year_MT 
import importlib # to reload the constants module. Necessary if we do changes in module 

### Import data

In [2]:
# import extended table, part 1 (session info)
# takes 2-3 minutes 
importlib.reload(constants_year_MT)
from constants_year_MT import url_12months_extended_part1
response_extended1 = requests.get(url_12months_extended_part1)
print(response_extended1)

# import extended table, part 2 (demographic info)
# takes 2-3 minutes
importlib.reload(constants_year_MT)
from constants_year_MT import url_12months_extended_part2
response_extended2 = requests.get(url_12months_extended_part2)
print(response_extended2)

<Response [200]>
<Response [200]>


### Create table from url data 

In [3]:
# function to create dataframe from url data 
def url_to_df (url, response): 
    """
    Function to create pandas dataframe from url data. Imported data needs 
    to be of type json or keyjson. Otherwise an exception error is thrown

    Arguments: 
        url: url previously imported from constants file 
        response: previously extracted response 

    Returns: 
        df2: supermetrics information transformed into pandas dataframe 
    """

    if 'data/json?' in url:
        # convert json into pandas df 
        data = json.loads(json.dumps(response.json()))
        df = pd.DataFrame.from_dict(data['data'])
        headers = df.iloc[0]
        df2 = pd.DataFrame(df.values[1:], columns=headers)
    elif 'data/keyjson?' in url:
        # convert json key-value pair output to pandas df 
        df2 = pd.read_json(json.dumps(response.json()))
    else:
        # throw exception message 
        import sys
        sys.exit("DEFINE JSON OR KEYJSON AS OUTPUT FORMAT: https://supermetrics.com/docs/product-api-output-formats/")
    return df2     

In [4]:
# create dataframe for year 2021 - extended set of features 
df_extended1 = url_to_df(url_12months_extended_part1, response_extended1)
df_extended2 = url_to_df(url_12months_extended_part2, response_extended2)
# join df_extended1 and df_extended2 using common features (specially clientID & Date)
df_extended = pd.merge(df_extended1, df_extended2, how = 'left', on = ['clientID', 'Date', 'Users', 'Transactions'])

### General data preprocessing  

* Transform data to correct format 
* Create feature of path order, indicating how many paths exist per clientID. To be used at attribution modelling
* Create feature of new/old client in 2021, using path_order

In [5]:
# function to convert data to correct format
def transform_class (df, feats_to_convert):
    for f in feats_to_convert: 
        if f == 'Date':
            df[f] = pd.to_datetime(df[f])
        else: # all numeric 
            df[f] = pd.to_numeric(df[f], errors = 'coerce')
    return df

In [6]:
feats_to_convert_for_extended = ['Date', 'Users', 'Transactions', 'Transaction revenue', 'Pageviews', 'Hits', 'Total time on site', 'Bounces', 'Total events', 'Latitude', 'Longitude']
df_extended = transform_class(df_extended, feats_to_convert_for_extended)

In [7]:
# create path order per clientID 
df_extended['path_order'] = df_extended.sort_values(['Date']).groupby(['clientID']).cumcount() + 1
# create feature of new/old client 
## new => path_order = 1
## old => path_order > 1
df_extended['new_old_client'] = (df_extended.path_order > 1).astype('int')
# change categories 
df_extended.new_old_client[df_extended.new_old_client == 1] = 'old_in_2021'
df_extended.new_old_client[df_extended.new_old_client == 0] = 'new_in_2021'

# ongoing? 

C:\Users\carol\AppData\Local\Temp\ipykernel_11416\3557078052.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_extended.new_old_client[df_extended.new_old_client == 1] = 'old_in_2021'


* 248.594 clients have just 1 touchpoint 
* 74.759 clients have at least 2 touchpoints; 48.244 have exactly 2 
* 26.515 clients have at least 3 touchpoints; 13.740 have exactly 2

(?) Does it make sense to work with those with just 1 touchpoint? Not sure about that yet 

### Data preprocessing for attribution modelling 

* We need to create a table that contains the path (order) for users that have interacted with different channels along the year 2021. 
* A reduced set of features was selected
* As we can observe, some entry errors probably exist - with very big path orders (which probably correspond to the same clientID)
* From the original ~ 350.000 unique clientIDS, 74.759 appear more than once in the dataset. We have to take into account that we are just checking 1 year timeframe - in other words, (potential) donors may have checked the webpage before 01.01.2021. That information should be contained in feature "user type", but Im not sure it I will do something with it

Example based on https://www.kaggle.com/code/hughhuyton/multitouch-attribution-modelling/notebook

(!) I am not 100% sure if that example is right 

In [8]:
# extract features of interest 
df_model = df_extended[['clientID', 'Date', 'Channel group', 'Operating system', 'Transactions', 'Transaction revenue']] # to work with this one 

# create path order per clientID 
df_model['path_order'] = df_model.sort_values(['Date']).groupby(['clientID']).cumcount() + 1

C:\Users\carol\AppData\Local\Temp\ipykernel_11416\195157793.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_model['path_order'] = df_model.sort_values(['Date']).groupby(['clientID']).cumcount() + 1


In [40]:
# Aggregate the channels a user interacted with into a single row 
df_paths = df_model.groupby('clientID')['Channel group'].agg(lambda x: x.tolist()).reset_index()
df_paths = df_paths.rename(columns={"Channel group": "path"})

# check frequency table 
df_paths.path.value_counts()
# df_paths.info()

[Organic Search]                                                                                                                                                                                                                                                                     93134
[Display]                                                                                                                                                                                                                                                                            48662
[Generic Paid Search]                                                                                                                                                                                                                                                                25753
[Direct]                                                                                                                                               

In [41]:
df_paths

,clientID,path
0,1000005671.1568444004,[Display]
1,1000006728.1622927843,[Direct]
2,1000020259.1556972396,[Referral]
3,1000023369.1635190743,[Referral]
4,1000024791.1630470945,[Branded Paid Search]
...,...,...
323348,999973068.1611254848,[Display]
323349,99997309.1608586309,[Display]
323350,999988377.1636720143,[Display]
323351,999996741.1623865085,[(Other)]


Check table for last-touch case

In [36]:
#Looking at the last interaction to see if it led to a user converting
df_model = df_model.sort_values(['clientID', 'path_order'])
df_last_interaction = df_model.drop_duplicates('clientID', keep='last')[['clientID', 'Transactions', 'Transaction revenue']]
df_last_interaction

,clientID,Date,path_order,Transactions,Transaction revenue
233994,1000005671.1568444004,2021-01-04,1,0,0.0
233995,1000006728.1622927843,2021-06-05,1,0,0.0
233996,1000020259.1556972396,2021-01-31,1,0,0.0
233997,1000023369.1635190743,2021-10-25,1,0,0.0
233998,1000024791.1630470945,2021-09-01,1,0,0.0
...,...,...,...,...,...
233990,999973068.1611254848,2021-01-21,1,0,0.0
23577,99997309.1608586309,2021-01-04,1,0,0.0
233991,999988377.1636720143,2021-11-12,1,0,0.0
233992,999996741.1623865085,2021-06-16,1,0,0.0


In [39]:
df_paths

,clientID,path,Transactions_x,Transaction revenue_x,Date_x,path_order_x,Transactions_y,Transaction revenue_y,Date_y,path_order_y,Transactions,Transaction revenue
0,1000005671.1568444004,Display,0,0.0,2021-01-04,1,0,0.0,2021-01-04,1,0,0.0
1,1000006728.1622927843,Direct,0,0.0,2021-06-05,1,0,0.0,2021-06-05,1,0,0.0
2,1000020259.1556972396,Referral,0,0.0,2021-01-31,1,0,0.0,2021-01-31,1,0,0.0
3,1000023369.1635190743,Referral,0,0.0,2021-10-25,1,0,0.0,2021-10-25,1,0,0.0
4,1000024791.1630470945,Branded Paid Search,0,0.0,2021-09-01,1,0,0.0,2021-09-01,1,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
323348,999973068.1611254848,Display,0,0.0,2021-01-21,1,0,0.0,2021-01-21,1,0,0.0
323349,99997309.1608586309,Display,0,0.0,2021-01-04,1,0,0.0,2021-01-04,1,0,0.0
323350,999988377.1636720143,Display,0,0.0,2021-11-12,1,0,0.0,2021-11-12,1,0,0.0
323351,999996741.1623865085,(Other),0,0.0,2021-06-16,1,0,0.0,2021-06-16,1,0,0.0


In [38]:
# merge with df_paths 
df_paths = pd.merge(df_paths, df_last_interaction, how = "left", on = "clientID")
df_paths

,clientID,path,Transactions_x,Transaction revenue_x,Date_x,path_order_x,Transactions_y,Transaction revenue_y,Date_y,path_order_y,Transactions,Transaction revenue
0,1000005671.1568444004,Display,0,0.0,2021-01-04,1,0,0.0,2021-01-04,1,0,0.0
1,1000006728.1622927843,Direct,0,0.0,2021-06-05,1,0,0.0,2021-06-05,1,0,0.0
2,1000020259.1556972396,Referral,0,0.0,2021-01-31,1,0,0.0,2021-01-31,1,0,0.0
3,1000023369.1635190743,Referral,0,0.0,2021-10-25,1,0,0.0,2021-10-25,1,0,0.0
4,1000024791.1630470945,Branded Paid Search,0,0.0,2021-09-01,1,0,0.0,2021-09-01,1,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
323348,999973068.1611254848,Display,0,0.0,2021-01-21,1,0,0.0,2021-01-21,1,0,0.0
323349,99997309.1608586309,Display,0,0.0,2021-01-04,1,0,0.0,2021-01-04,1,0,0.0
323350,999988377.1636720143,Display,0,0.0,2021-11-12,1,0,0.0,2021-11-12,1,0,0.0
323351,999996741.1623865085,(Other),0,0.0,2021-06-16,1,0,0.0,2021-06-16,1,0,0.0


In [26]:
df_paths['path'].value_counts()

Organic Search                                                                                                                                                                                                       93134
Display                                                                                                                                                                                                              48662
Generic Paid Search                                                                                                                                                                                                  25753
Direct                                                                                                                                                                                                               22953
Branded Paid Search                                                                                                         

Attribution models need paths to be limited by symbol ">" (like in R package)

In [13]:
# modify variable path 
def listToString(df):  
    str1 = ""  
    for i in df['path']:  
        str1 += i + ' > '    
    return str1[:-3]

df_paths['path'] = df_paths.apply(listToString, axis=1)
df_paths.head()

,clientID,path,Transactions,Transaction revenue
0,1000005671.1568444004,Display,0,0.0
1,1000006728.1622927843,Direct,0,0.0
2,1000020259.1556972396,Referral,0,0.0
3,1000023369.1635190743,Referral,0,0.0
4,1000024791.1630470945,Branded Paid Search,0,0.0


`df_paths` is a table that contains one row per unique clientID with features 
* `clientID`
* `path` - complete path, eventhough often just one touchpoint exists 
* `Transactions` 
* `Transaction revenue`

(?) Are transactions the total transactions for those clientIDs with more than one transaction? 

In [23]:
# grouped data of original dataset 
test1 = df_model.sort_values('clientID').groupby('clientID')['Transactions'].sum()
test1 = test1.to_frame()
test1

,Transactions
clientID,
1000005671.1568444004,0
1000006728.1622927843,0
1000020259.1556972396,0
1000023369.1635190743,0
1000024791.1630470945,0
...,...
999973068.1611254848,0
99997309.1608586309,0
999988377.1636720143,0


In [24]:
test2 = df_paths.sort_values('clientID')[['clientID', 'Transactions']]
test2

,clientID,Transactions
0,1000005671.1568444004,0
1,1000006728.1622927843,0
2,1000020259.1556972396,0
3,1000023369.1635190743,0
4,1000024791.1630470945,0
...,...,...
323348,999973068.1611254848,0
323349,99997309.1608586309,0
323350,999988377.1636720143,0
323351,999996741.1623865085,0


In [25]:
# compare test1 and test2 -> specially transactions 
## sum of transactions 
print(test1['Transactions'].sum())
print(test2['Transactions'].sum())

7725
5756


(!) Attention! I think `df_paths()` is not really correct, because clientIDs that had more than one transaction just appear once in the dataset. I think a pretreatment like we did in R would be necessary

## Heurstic approaches 

## Data-driven approaches. Markov Chains